In [1]:
import sys
sys.path.append("..")

In [2]:
from escalation.escalator import Escalator

In [4]:
import sys
sys.path.append("..")  # Move up one level to access the db/ folder

from db.connection import DBConnection

db = DBConnection(
    dbname="postgres",
    user="postgres",
    host="34.59.119.208",
    password="avantichhaya"
)

conn = db.connect()
if conn:
    cursor = conn.cursor()
else:
    print("🚫 DB connection failed. Cannot proceed.")

✅ Connected to PostgreSQL


In [5]:
from utils.constants import label_mapping, candidate_labels

print(candidate_labels)  # Just to test

['Requests related to taking leave or vacation', 'Questions about new hire onboarding or joining formalities', 'Inquiries regarding job offers or employment contracts', 'Questions related to salary, pay, wages, or payroll processing', 'Questions about insurance, medical, or employee benefits', 'Emails about resignation, exit process, or final settlements', 'Issues about work hours, attendance or timesheets', 'Questions about interview, screening or hiring stages', 'Clarification about company policies or procedures', 'Queries about training programs or skill development', 'Requests or updates regarding remote work', 'Inquiries about internal transfers or relocation', 'Questions about reimbursements or expense claims', 'Issues about system access, accounts, or technical problems', 'Emails about office events, parties, or celebrations']


In [6]:
escalator = Escalator(cursor=cursor, conn=conn, threshold=0.5)

In [7]:
from classifier.email_classifier import classify_email
from utils.constants import label_mapping

from classifier.email_classifier import classify_email

# Fetch unclassified emails
def fetch_unclassified_emails():
    cursor.execute("""
        SELECT email_id, subject, body
        FROM emails
        WHERE classified_category IS NULL
    """)
    return cursor.fetchall()

emails_to_classify = fetch_unclassified_emails()

for email_id, subject, body in emails_to_classify:
    predicted_label, confidence = classify_email(subject, body)

    if escalator.should_escalate(confidence):
        reason = f"Confidence below threshold: {confidence:.2f}"
        escalator.escalate(email_id, reason)
    else:
        cursor.execute("""
            UPDATE emails
            SET classified_category = %s
            WHERE email_id = %s
        """, (predicted_label, email_id))
        conn.commit()

    print(f"📩 Email ID {email_id}")
    print(f"🧠 Predicted: {predicted_label}")
    print(f"📈 Confidence: {round(confidence, 2)}\n")

print("✅ Classification + escalation complete.")

Device set to use cpu


✅ Classification + escalation complete.
